In [10]:
%matplotlib qt5
#%matplotlib inline
import numpy as np
import matplotlib.pylab as plt
import komm
from numpy.random import randint



SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.allclose
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


## Pulse shaping using Raised cosine pulse

In [15]:

samples_per_symbol = 16              # Upsampling factor
Nsym = 6                            # Filter span in symbol duration
alpha = 0.5                         # Roll-off factor

DataL = 20                              # Data length in symbols
R = 1000                                 # Data rate (how many samples per sec)
Fs = R*samples_per_symbol               # Sampling frequency

x_input = 2* randint(0,2,DataL)-1       # input random data
tx = 1000 * np.arange(0,DataL)/R        # time axis for the symbols (in msec)

# Filter the sequence of symbols with the filter
pulse = komm.RaisedCosinePulse(rolloff = alpha, length_in_symbols=Nsym)
imp = pulse.impulse_response #this vector has stored all time domain of pulses in all delays ..., h(t+1), h(t), h(t-1), h(t-2), ...
tx_filter = komm.TransmitFilter(pulse, samples_per_symbol=samples_per_symbol)

y0 = tx_filter(x_input)             #this is the signal y(t)
y0 = y0[:-samples_per_symbol*2] # we have to truncate the ending part

t0 = 1000 * np.arange(0,(DataL+Nsym/2)*samples_per_symbol)/Fs

# the following lines will filter group delay, since raised cosine filter is linear phase and symmetric
filtDelay = Nsym/(2*R)
y0 = y0[int(filtDelay*Fs):]
t0 = 1000 * np.arange(0,DataL*samples_per_symbol)/Fs



## Graph of the digital signal and the $y(t)$

In [16]:

# plot the digital signal
plt.figure
ax = plt.subplot(111)
ax.stem(tx,x_input, '-.')

# plot the pulse modified signal
ax= plt.subplot(111)
plt.plot(t0,y0)
plt.grid()
plt.title('Pulse shaping using raised cosine pulse')
plt.xlabel('Time (msec)')
plt.ylabel('$y(t)$')
plt.show()



## plotting the different pulses that form the resulting pulse

In [51]:
plt.figure
ax = plt.subplot(111)

for k, a in enumerate(x_input):
    ax.plot(t0, a*pulse.impulse_response(t0 - k))
ax.stem(tx,x_input, '-.')
ax.plot(t0,y0,'k', linewidth = 3)
plt.xlabel('Time (msec)')
plt.grid()
plt.title('Generation of the $y(t)$ pulse from individual raised cosine pulses')
plt.show()